In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
from collections import Counter
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
import time

In [143]:
df = pd.read_csv('paths_clean_CYF_Data.csv')

C:\Users\Anh Tran\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (421,422,423,433,434,611) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [144]:
df.head()

,id,client_id,case_number,assign_id,new,new_3,as_recnum,as_date_opened,as_date_closed,as_date_prev,as_date_next,as_length_IP,as_gap_between_IP,as_pop,as_loc,as_unit,as_subunit,as_name,as_close,svc_pre_all,svc_pre,svc_pre_sub,svc_during,svc_post_sub,svc_post,svc_post_all,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_op_7days_postdc,svc_op_30days_postdc,svc_op_60days_postdc,svc_op_90days_postdc,svc_es_7days_postdc,svc_es_30days_postdc,svc_es_60days_postdc,svc_es_90days_postdc,svc_ip_7days_postdc,svc_ip_30days_postdc,svc_ip_60days_postdc,svc_ip_90days_postdc,svc_dt_7days_postdc,svc_dt_30days_postdc,svc_dt_60days_postdc,svc_dt_90days_postdc,loc_pre_7days_ACT,loc_pre_30days_ACT,loc_pre_60days_ACT,loc_pre_90days_ACT,loc_pre_7days_CM,loc_pre_30days_CM,loc_pre_60days_CM,loc_pre_90days_CM,loc_pre_7days_CMINST,loc_pre_30days_CMINST,loc_pre_60days_CMINST,loc_pre_90days_CMINST,loc_pre_7days_CMSTR,loc_pre_30days_CMSTR,loc_pre_60days_CMSTR,loc_pre_90days_CMSTR,loc_pre_7days_CMTRAN,loc_pre_30days_CMTRAN,loc_pre_60days_CMTRAN,loc_pre_90days_CMTRAN,loc_pre_7days_WRAP,loc_pre_30days_WRAP,loc_pre_60days_WRAP,loc_pre_90days_WRAP,loc_pre_7days_TBS,loc_pre_30days_TBS,loc_pre_60days_TBS,loc_pre_90days_TBS,loc_pre_7days_FFS,loc_pre_30days_FFS,loc_pre_60days_FFS,loc_pre_90days_FFS,loc_pre_7days_OP,loc_pre_30days_OP,loc_pre_60days_OP,loc_pre_90days_OP,loc_pre_7days_OPLIHP,loc_pre_30days_OPLIHP,loc_pre_60days_OPLIHP,loc_pre_90days_OPLIHP,loc_pre_7days_OPR,loc_pre_30days_OPR,loc_pre_60days_OPR,loc_pre_90days_OPR,loc_pre_7days_PREV,loc_pre_30days_PREV,loc_pre_60days_PREV,loc_pre_90days_PREV,loc_pre_7days_CO,loc_pre_30days_CO,loc_pre_60days_CO,loc_pre_90days_CO,loc_pre_7days_CR,loc_pre_30days_CR,loc_pre_60days_CR,loc_pre_90days_CR,loc_pre_7days_CS,loc_pre_30days_CS,loc_pre_60days_CS,loc_pre_90days_CS,loc_pre_7days_ESU,loc_pre_30days_ESU,loc_pre_60days_ESU,loc_pre_90days_ESU,loc_pre_7days_PERT,loc_pre_30days_PERT,loc_pre_60days_PERT,loc_pre_90days_PERT,loc_pre_7days_UO,loc_pre_30days_UO,loc_pre_60days_UO,loc_pre_90days_UO,loc_pre_7days_IP,loc_pre_30days_IP,loc_pre_60days_IP,loc_pre_90days_IP,loc_pre_7days_IPCAPS,loc_pre_30days_IPCAPS,loc_pre_60days_IPCAPS,loc_pre_90days_IPCAPS,loc_pre_7days_IPCNTY,loc_pre_30days_IPCNTY,loc_pre_60days_IPCNTY,loc_pre_90days_IPCNTY,loc_pre_7days_IPFFS,loc_pre_30days_IPFFS,loc_pre_60days_IPFFS,loc_pre_90days_IPFFS,loc_pre_7days_IPLIHP,loc_pre_30days_IPLIHP,loc_pre_60days_IPLIHP,loc_pre_90days_IPLIHP,loc_pre_7days_IPST,loc_pre_30days_IPST,loc_pre_60days_IPST,loc_pre_90days_IPST,loc_pre_7days_JAIL,loc_pre_30days_JAIL,loc_pre_60days_JAIL,loc_pre_90days_JAIL,loc_pre_7days_JFS,loc_pre_30days_JFS,loc_pre_60days_JFS,loc_pre_90days_JFS,loc_pre_7days_EDG,loc_pre_30days_EDG,loc_pre_60days_EDG,loc_pre_90days_EDG,loc_pre_7days_LTC,loc_pre_30days_LTC,loc_pre_60days_LTC,loc_pre_90days_LTC,loc_pre_7days_LTCINST,loc_pre_30days_LTCINST,loc_pre_60days_LTCINST,loc_pre_90days_LTCINST,loc_pre_7days_LTCRES,loc_pre_30days_LTCRES,loc_pre_60days_LTCRES,loc_pre_90days_LTCRES,loc_pre_7days_RES,loc_pre_30days_RES,loc_pre_60days_RES,loc_pre_90days_RES,loc_pre_7days_BHC,loc_pre_30days_BHC,loc_pre_60days_BHC,loc_pre_90days_BHC,loc_pre_7days_DTC,loc_pre_30days_DTC,loc_pre_60days_DTC,loc_pre_90days_DTC,loc_pre_7days_DTCTF,loc_pre_30days_DTCTF,loc_pre_60days_DTCTF,loc_pre_90days_DTCTF,loc_pre_7days_DTR,loc_pre_30days_DTR,loc_pre_60days_DTR,loc_pre_90days_DTR,loc_pre_7days_STRTP,loc_pre_30days_STRTP,loc_pre_60days_STRTP,loc_pre_90days_STRTP,loc_post_7days_ACT,loc_post_30days_ACT,loc_post_60days_ACT,loc_post_90days_ACT,loc_post_7days_CM,loc_post_30days_CM,loc_post_60days_CM,loc_post_90days_CM,loc_post_7days_CMINST,loc_post_30days_CMINST,loc_post_60days_CMINST,loc_post_90days_CMINST,loc_post_7days_CMSTR,loc_post_30days_CMSTR,l

## Define variables used for the models

In [145]:

# Initial variables for the model. 
# I focus exclusively on the pre services and loc plus demographics with less than 20% of missing data.

id_cols = ['id', 'client_id']

demo_cols = ['age', 'age_group','race','gender','dx_group','lang', 'substance','insurance', 'liv', 'CWS', 'ADS','JJ']

ip_cols = ['as_loc', 'as_length_IP', 'IP_history', 'as_close']

loc_pre_cols = ['loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre']

svc_total_pre_cols = ['new', 'svc_pre_all', 'svc_pre', 'svc_pre_sub']

svc_CYF_timepoint_pre_cols = ['svc_op_7days_pre', 'svc_op_30days_pre', 'svc_op_60days_pre', 'svc_op_90days_pre',
                     'svc_es_7days_pre', 'svc_es_30days_pre', 'svc_es_60days_pre', 'svc_es_90days_pre',
                     'svc_dt_7days_pre', 'svc_dt_30days_pre', 'svc_dt_60days_pre', 'svc_dt_90days_pre',
                     'svc_ip_7days_pre', 'svc_ip_30days_pre', 'svc_ip_60days_pre', 'svc_ip_90days_pre']



output_cols = ['svc_CYF_ip_7days_postdc_bi', 'svc_CYF_ip_30days_postdc_bi', 'svc_CYF_ip_60days_postdc_bi', 'svc_CYF_ip_90days_postdc_bi']
input_cols = id_cols + demo_cols +  ip_cols + loc_pre_cols + svc_total_pre_cols  + svc_CYF_timepoint_pre_cols



In [146]:
df = df[input_cols + output_cols]
df.head()

,id,client_id,age,age_group,race,gender,dx_group,lang,substance,insurance,liv,CWS,ADS,JJ,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre,svc_CYF_ip_7days_postdc_bi,svc_CYF_ip_30days_postdc_bi,svc_CYF_ip_60days_postdc_bi,svc_CYF_ip_90days_postdc_bi
0,1,61,9,6-11 Yrs,Hispanic,Female,Anxiety Disorders,English,0.0,Medi-Cal Only,Foster Home,0.0,0.0,0.0,IPFFS,5,0.0,Home/Shelter/Self,OP,FFS,NaN,NaN,NaN,0,33,33,0,0,1,1,3,0,0,0,0,0,0,0,0,1,1,1,1,0,0,1,1
1,2,61,9,6-11 Yrs,Hispanic,Female,Anxiety Disorders,English,0.0,Medi-Cal Only,Foster Home,0.0,0.0,0.0,IPFFS,4,1.0,Home/Shelter/Self,TBS,FFS,OP,NaN,NaN,0,45,12,1,4,10,12,13,0,0,0,0,0,0,0,0,1,1,2,2,0,0,0,0
2,3,61,10,6-11 Yrs,Hispanic,Female,Anxiety Disorders,English,0.0,Medi-Cal Only,House or Apartment,0.0,0.0,0.0,IPFFS,3,2.0,Satisfactorily Achieved\r\nGoals,FFS,OP,PERT,NaN,NaN,0,122,77,1,2,5,11,13,0,1,1,1,0,0,0,0,1,1,1,1,0,0,1,1
3,4,61,10,6-11 Yrs,Hispanic,Female,Anxiety Disorders,English,0.0,Medi-Cal Only,House or Apartment,0.0,0.0,0.0,IPFFS,4,3.0,Satisfactorily Achieved\r\nGoals,FFS,OP,IPFFS,NaN,NaN,0,133,11,1,1,3,10,15,0,0,0,1,0,0,0,0,1,1,2,2,0,0,0,0
4,5,61,10,6-11 Yrs,Hispanic,Female,Anxiety Disorders,English,0.0,Medi-Cal Only,Residential Treatment Center,0.0,0.0,0.0,IPFFS,2,4.0,Satisfactorily Achieved\r\nGoals,FFS,OP,NaN,NaN,NaN,0,144,11,0,0,0,0,4,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0


In [147]:
# Recode some missing data
df.loc1_pre.fillna("None", inplace=True)
df.loc2_pre.fillna("None", inplace=True)
df.loc3_pre.fillna("None", inplace=True)
df.loc4_pre.fillna("None", inplace=True)
df.loc5_pre.fillna("None", inplace=True)


In [148]:
# Set a seed for reproducibility
SEED = 100
np.random.seed(SEED)
    
# create numpy arrays with the explanatory variables and output label
y = df[['id', 'svc_CYF_ip_7days_postdc_bi', 'svc_CYF_ip_30days_postdc_bi', 'svc_CYF_ip_60days_postdc_bi', 'svc_CYF_ip_90days_postdc_bi']]
X = df[input_cols]
    
# split the data into 80% train and 20% test, scale all variables between 0 and 1
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=SEED, stratify=y['svc_CYF_ip_90days_postdc_bi'])


In [149]:
Xtrain = Xtrain.sort_index()
Xtest = Xtest.sort_index()
ytrain = ytrain.sort_index()
ytest = ytest.sort_index()

In [150]:
ytrain.isin([np.nan, np.inf, -np.inf]).any()

id                             False
svc_CYF_ip_7days_postdc_bi     False
svc_CYF_ip_30days_postdc_bi    False
svc_CYF_ip_60days_postdc_bi    False
svc_CYF_ip_90days_postdc_bi    False
dtype: bool

In [151]:
# Recode categorical variables to numerical code - training dataset
original = Xtrain.copy()
mask = original.isnull()
Xtrain = Xtrain.astype(str).apply(LabelEncoder().fit_transform)
Xtrain = Xtrain.where(~mask, original)
Xtrain = Xtrain.astype('Int64')

mask = mask.sort_index()
original = original.sort_index()

In [152]:
# Recode categorical variables to numerical code - testing dataset
original2 = Xtest.copy()
mask2 = original2.isnull()
Xtest = Xtest.astype(str).apply(LabelEncoder().fit_transform)
Xtest = Xtest.where(~mask2, original2)
Xtest = Xtest.astype('Int64')

mask2 = mask2.sort_index()
original2 = original2.sort_index()

In [122]:
# Train dataset
print(original[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print(Xtrain[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print('##############')

print(original[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print(Xtrain[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print('##############')

print(original[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print(Xtrain[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print('##############')

print(original[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print(Xtrain[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print('##############')

print(original[['client_id','gender']].groupby('client_id').max().gender.value_counts())
print(Xtrain[['client_id','gender']].groupby('client_id').max().gender.value_counts())
print('##############')

Depressive Disorders                 2823
Bipolar Disorders                     547
Oppositional/Conduct Disorders        487
Schizophrenic Disorders               393
Other/ Excluded                       283
Anxiety Disorders                     217
Stressor and Adjustment Disorders     209
ADHD                                  139
Name: dx_group, dtype: int64
3    2855
2     563
4     496
6     399
5     290
1     219
7     214
0     141
Name: dx_group, dtype: Int64
##############
English                     4714
Spanish                      523
Other/Unknown                 98
Middle Eastern languages      29
Asian languages               25
Name: lang, dtype: int64
1    4733
4     523
3      99
2      29
0      25
Name: lang, dtype: Int64
##############
Medi-Cal Only            4639
Other Insurance           332
Any Private Insurance     210
Uninsured/Unknown         208
Name: insurance, dtype: int64
1    4655
2     333
0     212
3     209
Name: insurance, dtype: Int64
###########

In [123]:
# Test dataset
print(original2[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print(Xtest[['client_id','dx_group']].groupby('client_id').max().dx_group.value_counts())
print('##############')

print(original2[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print(Xtest[['client_id','lang']].groupby('client_id').max().lang.value_counts())
print('##############')

print(original2[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print(Xtest[['client_id','insurance']].groupby('client_id').max().insurance.value_counts())
print('##############')


print(original2[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print(Xtest[['client_id','liv']].groupby('client_id').max().liv.value_counts())
print('##############')

print(original2[['client_id','gender']].groupby('client_id').max().gender.value_counts())
print(Xtest[['client_id','gender']].groupby('client_id').max().gender.value_counts())
print('##############')

Depressive Disorders                 978
Bipolar Disorders                    242
Oppositional/Conduct Disorders       210
Schizophrenic Disorders              163
Other/ Excluded                      106
Anxiety Disorders                     83
Stressor and Adjustment Disorders     70
ADHD                                  42
Name: dx_group, dtype: int64
3    984
2    243
4    212
6    164
5    108
1     84
7     70
0     42
Name: dx_group, dtype: Int64
##############
English                     1766
Spanish                      169
Other/Unknown                 32
Asian languages               10
Middle Eastern languages       4
Name: lang, dtype: int64
1    1769
4     169
3      33
0      10
2       4
Name: lang, dtype: Int64
##############
Medi-Cal Only            1719
Other Insurance           120
Any Private Insurance      75
Uninsured/Unknown          67
Name: insurance, dtype: int64
1    1722
2     120
0      76
3      67
Name: insurance, dtype: Int64
##############
House or Apa

### Data imputation for training dataset

In [153]:
# Check data type
var_name = list(Xtrain.columns.values)
type_name = list(Xtrain.dtypes)
var_list = pd.DataFrame(
    {'variable_name': var_name,
     'data_type': type_name
    })
pd.set_option('display.max_rows', None)
print(var_list)

        variable_name data_type
0                  id     Int64
1           client_id     Int64
2                 age     Int64
3           age_group     Int64
4                race     Int64
5              gender     Int64
6            dx_group     Int64
7                lang     Int64
8           substance     Int64
9           insurance     Int64
10                liv     Int64
11                CWS     Int64
12                ADS     Int64
13                 JJ     Int64
14             as_loc     Int64
15       as_length_IP     Int64
16         IP_history     Int64
17           as_close     Int64
18           loc1_pre     Int64
19           loc2_pre     Int64
20           loc3_pre     Int64
21           loc4_pre     Int64
22           loc5_pre     Int64
23                new     Int64
24        svc_pre_all     Int64
25            svc_pre     Int64
26        svc_pre_sub     Int64
27   svc_op_7days_pre     Int64
28  svc_op_30days_pre     Int64
29  svc_op_60days_pre     Int64
30  svc_

In [154]:
# count number of rows with missing values
n_miss = Xtrain.isnull().sum()
perc = Xtrain.isnull().sum()/len(Xtrain)*100
df_missing = pd.merge(n_miss.rename('n_miss'), perc.rename('perc_miss'), right_index = True,
               left_index = True)
pd.set_option('display.max_rows', None)
print(df_missing)

                   n_miss  perc_miss
id                      0   0.000000
client_id               0   0.000000
age                     0   0.000000
age_group               0   0.000000
race                    0   0.000000
gender                153   1.368760
dx_group              541   4.839864
lang                  153   1.368760
substance             153   1.368760
insurance             153   1.368760
liv                   153   1.368760
CWS                   153   1.368760
ADS                   153   1.368760
JJ                    153   1.368760
as_loc                  0   0.000000
as_length_IP            0   0.000000
IP_history              0   0.000000
as_close                0   0.000000
loc1_pre                0   0.000000
loc2_pre                0   0.000000
loc3_pre                0   0.000000
loc4_pre                0   0.000000
loc5_pre                0   0.000000
new                     0   0.000000
svc_pre_all             0   0.000000
svc_pre                 0   0.000000
s

In [155]:
mis_dat_cols = ['gender', 'dx_group','lang','insurance','liv','substance', 'CWS', 'ADS', 'JJ']

In [156]:
pd.set_option('display.max_columns', None)
Xtrain.head()

,id,client_id,age,age_group,race,gender,dx_group,lang,substance,insurance,liv,CWS,ADS,JJ,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre
0,0,4751,33,4,2,0,1,1,0,1,2,0,0,0,2,54,0,2,19,10,12,13,9,0,456,267,0,0,1,1,84,0,0,0,0,0,0,0,0,0,0,0,0
1,4038,4751,33,4,2,0,1,1,0,1,2,0,0,0,2,42,1,2,24,10,13,13,9,0,586,41,1,29,2,18,34,0,0,0,0,0,0,0,0,0,0,11,11
2,4947,4751,0,4,2,0,1,1,0,1,5,0,0,0,2,30,12,11,11,17,15,13,9,0,115,479,1,12,52,11,34,0,1,1,1,0,0,0,0,0,0,0,0
3,5827,4751,0,4,2,0,1,1,0,1,5,0,0,0,2,42,23,11,11,17,10,13,9,0,147,22,1,1,30,2,50,0,0,0,1,0,0,0,0,0,0,11,11
6,8500,4751,2,1,2,0,1,1,0,1,7,0,0,1,1,63,35,11,9,17,12,13,9,0,442,39,0,0,1,91,129,23,26,30,31,0,12,11,40,0,0,0,0


In [157]:
Xtrain = Xtrain.drop(['id', 'client_id'], axis = 1)

In [158]:
Xtrain.shape[0]

11178

In [159]:
k = math.sqrt(Xtrain.shape[0])/2
k

52.86303056011829

In [160]:
#Imputing data with KNN
imputer = KNNImputer(n_neighbors=53, weights='uniform', metric='nan_euclidean')
Xtrain = pd.DataFrame(imputer.fit_transform(Xtrain),columns = Xtrain.columns)

In [161]:
Xtrain = Xtrain.astype(int)
Xtrain.head()

,age,age_group,race,gender,dx_group,lang,substance,insurance,liv,CWS,ADS,JJ,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre
0,33,4,2,0,1,1,0,1,2,0,0,0,2,54,0,2,19,10,12,13,9,0,456,267,0,0,1,1,84,0,0,0,0,0,0,0,0,0,0,0,0
1,33,4,2,0,1,1,0,1,2,0,0,0,2,42,1,2,24,10,13,13,9,0,586,41,1,29,2,18,34,0,0,0,0,0,0,0,0,0,0,11,11
2,0,4,2,0,1,1,0,1,5,0,0,0,2,30,12,11,11,17,15,13,9,0,115,479,1,12,52,11,34,0,1,1,1,0,0,0,0,0,0,0,0
3,0,4,2,0,1,1,0,1,5,0,0,0,2,42,23,11,11,17,10,13,9,0,147,22,1,1,30,2,50,0,0,0,1,0,0,0,0,0,0,11,11
4,2,1,2,0,1,1,0,1,7,0,0,1,1,63,35,11,9,17,12,13,9,0,442,39,0,0,1,91,129,23,26,30,31,0,12,11,40,0,0,0,0


In [162]:
Xtrain.isnull().values.any()

False

In [163]:
Xtrain_imputedcols = Xtrain[mis_dat_cols]
Xtrain_othercols = original.drop([x for x in mis_dat_cols], axis = 1)
Xtrain_imputedcols = Xtrain_imputedcols.reset_index(drop=True)
Xtrain_othercols = Xtrain_othercols.reset_index(drop=True)

In [164]:
Xtrain_copy = Xtrain.copy()
df2 = Xtrain_othercols.join(Xtrain_imputedcols)
print(df2.shape)
print(df2.isnull().values.any())

(11178, 43)
False


In [165]:
df2['dx_group'] = df2['dx_group'].replace(0,'ADHD')
df2['dx_group'] = df2['dx_group'].replace(1,'Anxiety Disorders')
df2['dx_group'] = df2['dx_group'].replace(2,'Bipolar Disorders')
df2['dx_group'] = df2['dx_group'].replace(3,'Depressive Disorders')
df2['dx_group'] = df2['dx_group'].replace(4,'Oppositional/Conduct Disorders')
df2['dx_group'] = df2['dx_group'].replace(5,'Other/ Excluded')
df2['dx_group'] = df2['dx_group'].replace(6,'Schizophrenic Disorders')
df2['dx_group'] = df2['dx_group'].replace(7,'Stressor and Adjustment Disorders')

df2['lang'] = df2['lang'].replace(0,'Asian languages')
df2['lang'] = df2['lang'].replace(1,'English')
df2['lang'] = df2['lang'].replace(2,'Middle Eastern languages')
df2['lang'] = df2['lang'].replace(3,'Other/Unknown')
df2['lang'] = df2['lang'].replace(4,'Spanish')

df2['insurance'] = df2['insurance'].replace(0,'Medi-Cal Only')
df2['insurance'] = df2['insurance'].replace(1,'Other Insurance')
df2['insurance'] = df2['insurance'].replace(2,'Private')
df2['insurance'] = df2['insurance'].replace(3,'Uninsured/Unknown')


df2['liv'] = df2['liv'].replace(0,'Children\'s Shelter')
df2['liv'] = df2['liv'].replace(1,'Correctional Facility')
df2['liv'] = df2['liv'].replace(2,'Foster Home')
df2['liv'] = df2['liv'].replace(3,'Group Home')
df2['liv'] = df2['liv'].replace(4,'Homeless')
df2['liv'] = df2['liv'].replace(5,'House or Apartment')
df2['liv'] = df2['liv'].replace(6,'Lives Independently')
df2['liv'] = df2['liv'].replace(7,'Other')
df2['liv'] = df2['liv'].replace(8,'Unknown')

df2['gender'] = df2['gender'].replace(0,'Female')
df2['gender'] = df2['gender'].replace(1,'Male')
df2['gender'] = df2['gender'].replace(2,'Other/Unknown')


In [166]:
df2.head()

,id,client_id,age,age_group,race,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre,gender,dx_group,lang,insurance,liv,substance,CWS,ADS,JJ
0,1,61,9,6-11 Yrs,Hispanic,IPFFS,5,0.0,Home/Shelter/Self,OP,FFS,None,None,None,0,33,33,0,0,1,1,3,0,0,0,0,0,0,0,0,1,1,1,1,Female,Anxiety Disorders,English,Other Insurance,Foster Home,0,0,0,0
1,2,61,9,6-11 Yrs,Hispanic,IPFFS,4,1.0,Home/Shelter/Self,TBS,FFS,OP,None,None,0,45,12,1,4,10,12,13,0,0,0,0,0,0,0,0,1,1,2,2,Female,Anxiety Disorders,English,Other Insurance,Foster Home,0,0,0,0
2,3,61,10,6-11 Yrs,Hispanic,IPFFS,3,2.0,Satisfactorily Achieved\r\nGoals,FFS,OP,PERT,None,None,0,122,77,1,2,5,11,13,0,1,1,1,0,0,0,0,1,1,1,1,Female,Anxiety Disorders,English,Other Insurance,House or Apartment,0,0,0,0
3,4,61,10,6-11 Yrs,Hispanic,IPFFS,4,3.0,Satisfactorily Achieved\r\nGoals,FFS,OP,IPFFS,None,None,0,133,11,1,1,3,10,15,0,0,0,1,0,0,0,0,1,1,2,2,Female,Anxiety Disorders,English,Other Insurance,House or Apartment,0,0,0,0
4,7,61,12,12-15 Yrs,Hispanic,IPCAPS,6,6.0,Satisfactorily Achieved\r\nGoals,DTR,OP,None,None,None,0,317,119,0,0,1,6,7,3,3,3,3,0,2,18,36,1,1,1,1,Female,Anxiety Disorders,English,Other Insurance,Other,0,0,0,1


In [167]:
df2.to_csv (r'paths_clean2_CYF_train.csv', index = False, header=True)

In [168]:
#Xtrain = df2
Xtrain = pd.read_csv('paths_clean2_CYF_train.csv')

In [169]:
Xtrain.isnull().values.any()

False

### Data imputation for testing dataset

In [170]:
# count number of rows with missing values
n_miss = Xtest.isnull().sum()
perc = Xtest.isnull().sum()/len(Xtest)*100
df_missing = pd.merge(n_miss.rename('n_miss'), perc.rename('perc_miss'), right_index = True,
               left_index = True)
pd.set_option('display.max_rows', None)
print(df_missing)

                   n_miss  perc_miss
id                      0   0.000000
client_id               0   0.000000
age                     0   0.000000
age_group               0   0.000000
race                    0   0.000000
gender                 39   1.395349
dx_group              131   4.686941
lang                   39   1.395349
substance              39   1.395349
insurance              39   1.395349
liv                    39   1.395349
CWS                    39   1.395349
ADS                    39   1.395349
JJ                     39   1.395349
as_loc                  0   0.000000
as_length_IP            0   0.000000
IP_history              0   0.000000
as_close                0   0.000000
loc1_pre                0   0.000000
loc2_pre                0   0.000000
loc3_pre                0   0.000000
loc4_pre                0   0.000000
loc5_pre                0   0.000000
new                     0   0.000000
svc_pre_all             0   0.000000
svc_pre                 0   0.000000
s

In [171]:
mis_dat_cols = ['gender','dx_group','lang','insurance','liv','substance', 'CWS', 'ADS', 'JJ']

In [172]:
Xtest = Xtest.drop(['id', 'client_id'], axis = 1)

In [173]:
k = math.sqrt(Xtest.shape[0])/2
k

26.433879775772606

In [174]:
#Imputing data with KNN
imputer = KNNImputer(n_neighbors=27, weights='uniform', metric='nan_euclidean')
Xtest = pd.DataFrame(imputer.fit_transform(Xtest),columns = Xtest.columns)

In [175]:
Xtest = Xtest.astype(int)
Xtest.isnull().values.any()

False

In [176]:
Xtest_imputedcols = Xtest[mis_dat_cols]
Xtest_othercols = original2.drop([x for x in mis_dat_cols], axis = 1)
Xtest_imputedcols = Xtest_imputedcols.reset_index(drop=True)
Xtest_othercols = Xtest_othercols.reset_index(drop=True)

In [177]:
Xtest_copy = Xtest.copy()
df3 = Xtest_othercols.join(Xtest_imputedcols)
print(df3.shape)
print(df3.isnull().values.any())

(2795, 43)
False


In [178]:
df3['dx_group'] = df3['dx_group'].replace(0,'ADHD')
df3['dx_group'] = df3['dx_group'].replace(1,'Anxiety Disorders')
df3['dx_group'] = df3['dx_group'].replace(2,'Bipolar Disorders')
df3['dx_group'] = df3['dx_group'].replace(3,'Depressive Disorders')
df3['dx_group'] = df3['dx_group'].replace(4,'Oppositional/Conduct Disorders')
df3['dx_group'] = df3['dx_group'].replace(5,'Other/ Excluded')
df3['dx_group'] = df3['dx_group'].replace(6,'Schizophrenic Disorders')
df3['dx_group'] = df3['dx_group'].replace(7,'Stressor and Adjustment Disorders')

df3['lang'] = df3['lang'].replace(0,'Asian languages')
df3['lang'] = df3['lang'].replace(1,'English')
df3['lang'] = df3['lang'].replace(2,'Middle Eastern languages')
df3['lang'] = df3['lang'].replace(3,'Other/Unknown')
df3['lang'] = df3['lang'].replace(4,'Spanish')

df3['insurance'] = df3['insurance'].replace(0,'Medi-Cal Only')
df3['insurance'] = df3['insurance'].replace(1,'Other Insurance')
df3['insurance'] = df3['insurance'].replace(2,'Private')
df3['insurance'] = df3['insurance'].replace(3,'Uninsured/Unknown')


df3['liv'] = df3['liv'].replace(0,'Children\'s Shelter')
df3['liv'] = df3['liv'].replace(1,'Correctional Facility')
df3['liv'] = df3['liv'].replace(2,'Foster Home')
df3['liv'] = df3['liv'].replace(3,'Group Home')
df3['liv'] = df3['liv'].replace(4,'Homeless')
df3['liv'] = df3['liv'].replace(5,'House or Apartment')
df3['liv'] = df3['liv'].replace(6,'Lives Independently')
df3['liv'] = df3['liv'].replace(7,'Other')
df3['liv'] = df3['liv'].replace(8,'Unknown')


df3['gender'] = df3['gender'].replace(0,'Female')
df3['gender'] = df3['gender'].replace(1,'Male')
df3['gender'] = df3['gender'].replace(2,'Other/Unknown')
                                  

In [179]:
df3.head()

,id,client_id,age,age_group,race,as_loc,as_length_IP,IP_history,as_close,loc1_pre,loc2_pre,loc3_pre,loc4_pre,loc5_pre,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre,gender,dx_group,lang,insurance,liv,substance,CWS,ADS,JJ
0,5,61,10,6-11 Yrs,Hispanic,IPFFS,2,4.0,Satisfactorily Achieved\r\nGoals,FFS,OP,None,None,None,0,144,11,0,0,0,0,4,0,0,0,0,0,0,0,0,1,1,1,1,Female,Anxiety Disorders,English,Other Insurance,Other,0,0,0,0
1,6,61,11,6-11 Yrs,Hispanic,IPFFS,2,5.0,Satisfactorily Achieved\r\nGoals,OP,None,None,None,None,0,198,54,1,1,7,11,19,0,0,1,1,0,0,0,0,1,1,1,1,Female,Anxiety Disorders,English,Other Insurance,Other,0,0,0,0
2,8,61,13,12-15 Yrs,Hispanic,IPFFS,5,7.0,Crisis Residence,DTR,TBS,None,None,None,0,522,205,0,1,9,17,19,0,0,0,0,5,24,46,64,1,1,1,1,Female,Bipolar Disorders,English,Other Insurance,Correctional Facility,0,0,0,1
3,12,61,15,12-15 Yrs,Hispanic,IPFFS,5,11.0,Home/Shelter/Self,WRAP,OP,UO,None,None,0,632,71,0,6,27,56,77,0,1,2,2,0,0,0,0,1,1,1,2,Female,Bipolar Disorders,English,Other Insurance,House or Apartment,0,1,0,1
4,13,61,15,12-15 Yrs,Hispanic,IPFFS,1,12.0,Home/Shelter/Self,WRAP,OP,None,None,None,0,716,84,0,13,38,71,95,0,0,1,2,0,0,0,0,1,1,1,2,Female,Bipolar Disorders,English,Other Insurance,House or Apartment,0,1,0,1


In [180]:
df3.to_csv (r'paths_clean2_CYF_test.csv', index = False, header=True)

In [181]:
#Xtest = df3
Xtest = pd.read_csv('paths_clean2_CYF_test.csv')

In [182]:
Xtest.isnull().values.any()

False

### Define features for the models

### Numerical features:
1. Age ('age')
2. Length since previous hospitalization ('as_gap_since_former_IP')
3. Length of hospital stay ('as_length_IP')
4. Number of previous hospitalizations ('IP_history')
5. Number of all services before index hospitalization ('svc_pre_all')
6. Number of services before index hospitalization since previous hospitalization ('svc_pre')
7. Number of LOC received before index hospitalization by LOC type and timepoint

In [183]:
numerical_features = ['age', 'as_length_IP', 'IP_history', 'svc_pre_all', 'svc_pre', 'svc_pre_sub']
numerical_features = numerical_features + svc_CYF_timepoint_pre_cols

In [184]:
len(numerical_features)

22

### Categorical features:
1. Demographics (except age): 'age_group','race','gender','dx_group','lang', 'substance','insurance','emp', 'educ','liv','justice', 'sexorient'
2. Type of IP ('as_loc')
3. Type of discharge ('as_close')
4. Last 5 LOCs before index hospitalization
5. Any services before index hospitalization ('new')
6. Any services for 3 years before index hospitalization ('new_3')

In [185]:
# Create dummy variables for non-binary categorical variables
Xtrain = pd.get_dummies(Xtrain, columns=['age_group','race','gender','dx_group','lang', 'insurance', 'liv', 
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

In [186]:
pd.set_option('display.max_columns', None)
Xtrain.head()

,id,client_id,age,as_length_IP,IP_history,new,svc_pre_all,svc_pre,svc_pre_sub,svc_op_7days_pre,svc_op_30days_pre,svc_op_60days_pre,svc_op_90days_pre,svc_es_7days_pre,svc_es_30days_pre,svc_es_60days_pre,svc_es_90days_pre,svc_dt_7days_pre,svc_dt_30days_pre,svc_dt_60days_pre,svc_dt_90days_pre,svc_ip_7days_pre,svc_ip_30days_pre,svc_ip_60days_pre,svc_ip_90days_pre,substance,CWS,ADS,JJ,age_group_12-15 Yrs,age_group_16-17 Yrs,age_group_18+ Yrs,age_group_6-11 Yrs,race_Black,race_Hispanic,race_Native American,race_Other,race_Unknown/NR,race_White,gender_Male,gender_Other/Unknown,dx_group_Anxiety Disorders,dx_group_Bipolar Disorders,dx_group_Depressive Disorders,dx_group_Oppositional/Conduct Disorders,dx_group_Other/ Excluded,dx_group_Schizophrenic Disorders,dx_group_Stressor and Adjustment Disorders,lang_English,lang_Middle Eastern languages,lang_Other/Unknown,lang_Spanish,insurance_Other Insurance,insurance_Private,insurance_Uninsured/Unknown,liv_Correctional Facility,liv_Foster Home,liv_Group Home,liv_Homeless,liv_House or Apartment,liv_Lives Independently,liv_Other,liv_Unknown,as_loc_IPCAPS,as_loc_IPFFS,as_loc_PHF,as_close_Crisis Residence,as_close_Home/Shelter/Self,as_close_Homeless,as_close_Justice Involved,as_close_Medical Hospital,as_close_Other,as_close_Psychiatric Hospital,as_close_Required Higher Level\r\nof Care,as_close_Required Lower Level\r\nof Care,as_close_Required Same Level\r\nof Care,as_close_Satisfactorily Achieved\r\nGoals,as_close_Unknown,loc1_pre_CM,loc1_pre_CMINST,loc1_pre_CMSTR,loc1_pre_CR,loc1_pre_CS,loc1_pre_DT,loc1_pre_DTC,loc1_pre_DTCTF,loc1_pre_DTR,loc1_pre_ESU,loc1_pre_FFS,loc1_pre_IP,loc1_pre_IPCAPS,loc1_pre_IPFFS,loc1_pre_IPLIHP,loc1_pre_JAIL,loc1_pre_JFS,loc1_pre_None,loc1_pre_OP,loc1_pre_OPR,loc1_pre_PERT,loc1_pre_PREV,loc1_pre_STRTP,loc1_pre_TBS,loc1_pre_UO,loc1_pre_WRAP,loc2_pre_CM,loc2_pre_CMTRAN,loc2_pre_CR,loc2_pre_CS,loc2_pre_DT,loc2_pre_DTC,loc2_pre_DTCTF,loc2_pre_DTR,loc2_pre_ESU,loc2_pre_FFS,loc2_pre_IP,loc2_pre_IPCAPS,loc2_pre_IPFFS,loc2_pre_JAIL,loc2_pre_JFS,loc2_pre_None,loc2_pre_OP,loc2_pre_OPR,loc2_pre_PERT,loc2_pre_PREV,loc2_pre_STRTP,loc2_pre_TBS,loc2_pre_UO,loc2_pre_WRAP,loc3_pre_CR,loc3_pre_CS,loc3_pre_DTC,loc3_pre_DTR,loc3_pre_ESU,loc3_pre_FFS,loc3_pre_IP,loc3_pre_IPCAPS,loc3_pre_IPCNTY,loc3_pre_IPFFS,loc3_pre_JFS,loc3_pre_None,loc3_pre_OP,loc3_pre_OPR,loc3_pre_PERT,loc3_pre_PREV,loc3_pre_RES,loc3_pre_STRTP,loc3_pre_TBS,loc3_pre_UO,loc3_pre_WRAP,loc4_pre_CR,loc4_pre_CS,loc4_pre_DTC,loc4_pre_DTR,loc4_pre_ESU,loc4_pre_FFS,loc4_pre_IP,loc4_pre_IPCAPS,loc4_pre_IPCNTY,loc4_pre_IPFFS,loc4_pre_JAIL,loc4_pre_JFS,loc4_pre_None,loc4_pre_OP,loc4_pre_OPR,loc4_pre_PERT,loc4_pre_PREV,loc4_pre_STRTP,loc4_pre_TBS,loc4_pre_UO,loc4_pre_WRAP,loc5_pre_CR,loc5_pre_CS,loc5_pre_DTC,loc5_pre_DTR,loc5_pre_FFS,loc5_pre_IP,loc5_pre_IPCAPS,loc5_pre_IPFFS,loc5_pre_None,loc5_pre_OP,loc5_pre_OPR,loc5_pre_PERT,loc5_pre_PREV,loc5_pre_TBS,loc5_pre_UO,loc5_pre_WRAP
0,1,61,9,5,0.0,0,33,33,0,0,1,1,3,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,2,61,9,4,1.0,0,45,12,1,4,10,12,13,0,0,0,0,0,0,0,0,1,1,2,2,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,3,61,10,3,2.0,0,122,77,1,2,5,11,13,0,1,1,1,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [187]:
Xtrain.shape

(11178, 186)

In [189]:
dummy_vars = Xtrain.loc[:, 'age_group_12-15 Yrs':'loc5_pre_WRAP']
dummy_vars = list(dummy_vars.columns)
categorical_features = dummy_vars + ['substance', 'new','CWS', 'ADS', 'JJ']

In [190]:
# Select all features
all_features = numerical_features + categorical_features + ['id', 'client_id']
len(all_features)

186

In [191]:
# Create dummy variables for non-binary categorical variables
Xtest = pd.get_dummies(Xtest, columns=['age_group','race','gender','dx_group','lang', 'insurance','liv', 
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

In [192]:
Xtest.shape

(2795, 169)

In [193]:
Xtest_extra_cols = [x for x in all_features if x not in Xtest.columns]
Xtest_extra_cols

['loc1_pre_CMINST',
 'loc1_pre_DT',
 'loc1_pre_IPLIHP',
 'loc1_pre_JAIL',
 'loc2_pre_CM',
 'loc2_pre_DTCTF',
 'loc2_pre_JAIL',
 'loc3_pre_CR',
 'loc3_pre_ESU',
 'loc3_pre_IPCNTY',
 'loc3_pre_RES',
 'loc4_pre_CR',
 'loc4_pre_ESU',
 'loc4_pre_IPCNTY',
 'loc4_pre_JAIL',
 'loc4_pre_STRTP',
 'loc4_pre_UO',
 'loc5_pre_CR',
 'loc5_pre_CS',
 'loc5_pre_IP',
 'loc5_pre_PREV']

In [194]:
Xtest_extra_cols_df = pd.DataFrame(0, index=np.arange(Xtest.shape[0]), columns=Xtest_extra_cols)

In [195]:
Xtest =Xtest.join(Xtest_extra_cols_df)
Xtest.shape

(2795, 190)

In [196]:
Xtrain_extra_cols = [x for x in Xtest.columns if x not in Xtrain.columns]
Xtrain_extra_cols
Xtrain_extra_cols_df = pd.DataFrame(0, index=np.arange(Xtrain.shape[0]), columns=Xtrain_extra_cols)
Xtrain =Xtrain.join(Xtrain_extra_cols_df)
Xtrain.shape

(11178, 190)

In [197]:
name_cols = list(Xtrain.columns)
df_train = pd.merge(Xtrain, ytrain, on='id')
df_test = pd.merge(Xtest, ytest, on='id')

In [198]:
df_train.to_csv (r'paths_CYF_trainData.csv', index = False, header=True)
df_test.to_csv (r'paths_CYF_testData.csv', index = False, header=True)

### Archive codes

In [ ]:
'''
# Map values
dx_group_dict = {0:'Anxiety Disorders', 1:'Bipolar Disorders', 2:'Depressive Disorders', 3: 'Other/ Excluded',
                4: 'Schizophrenia and Other Psychotic Disorders', 5: 'Stressor and Adjustment Disorders'}
lang_dict = {0:'Asian languages', 1:'English', 2:'Middle Eastern languages', 3:'Other/Unknown', 4:'Spanish'}
insurance_dict = {0:'Medi-Cal + Medicare', 1:'Private', 2:'Uninsured/Unknown'}
emp_dict = {0:'Competitive Job', 1:'Not Seeking Work', 2:'Not in Labor Force', 3:'Other', 4:'Resident/Inmate of Institution',
           5:'Seeking Work', 6:'Unknown'}
educ_dict = {0:'Associates Degree', 1:'Bachelors Degree', 2:'High School Diploma/GED', 3:'High School Not Completed',
            4:'Masters Degree', 5:'Some College/Vocational Training', 6:'Unknown/Not Reported'}
liv_dict = {0:'Board & Care', 1:'Homeless', 2:'Institutional', 3:'Justice Related', 4:'Lives Independently', 5:'Other/Unknown'}
sexorient_dict = {0:'Bisexual', 1:'Decline to State', 2:'Deferred', 3:'Gay Male', 4:'Heterosexual', 5:'Lesbian',
                 6:'Missing', 7:'Other', 8:'Questioning', 9:'Transgender'}
df2['dx_group'] = df2['dx_group'].map(dx_group_dict)
df2['lang'] = df2['lang'].map(lang_dict)
df2['insurance'] = df2['insurance'].map(insurance_dict)
df2['emp'] = df2['emp'].map(emp_dict)
df2['educ'] = df2['educ'].map(educ_dict)
df2['liv'] = df2['liv'].map(liv_dict)
df2['sexorient'] = df2['sexorient'].map(sexorient_dict)
'''

In [ ]:
'''
#Impute missing value with mean
for col in numerical_features:
    df[col].fillna(df[col].mean(), inplace=True)
'''

In [ ]:
'''
# Create dummy variables for non-binary categorical variables
df = pd.get_dummies(df, columns=['age_group','race','gender','dx_group','lang', 'insurance','emp', 'educ','liv', 'sexorient',
                       'as_loc', 'as_close','loc1_pre', 'loc2_pre', 'loc3_pre', 'loc4_pre', 'loc5_pre'], drop_first=True)

'''

In [ ]:
'''
# impute missing values of categorical features with the most frequent object
for col in categorical_features:
    df[col].fillna(df[col].value_counts().index[0], inplace=True)
'''

In [ ]:
'''
svc_AOA_timepoint_pre_cols = [ 'svc_AOA_op_7days_pre', 'svc_AOA_op_30days_pre', 'svc_AOA_op_60days_pre', 'svc_AOA_op_90days_pre',
                     'svc_AOA_es_7days_pre', 'svc_AOA_es_30days_pre', 'svc_AOA_es_60days_pre', 'svc_AOA_es_90days_pre',
                     'svc_AOA_fs_7days_pre', 'svc_AOA_fs_30days_pre', 'svc_AOA_fs_60days_pre', 'svc_AOA_fs_90days_pre',
                     'svc_AOA_24h_7days_pre', 'svc_AOA_24h_30days_pre', 'svc_AOA_24h_60days_pre', 'svc_AOA_24h_90days_pre',
                     'svc_AOA_ip_7days_pre', 'svc_AOA_ip_30days_pre', 'svc_AOA_ip_60days_pre', 'svc_AOA_ip_90days_pre']

'''

In [ ]:
'''
# Calculate the prevalence of rehospitalization
admissions = len(df)
counts = df.groupby('output')['id'].count().values
prevalence = counts[1]/(counts[0]+counts[1])
print("The total hospitalization is ", admissions)
print("The prevalence of 90-day rehospitalization is ", prevalence)
print("The absolute number of rehospitalized patients is ", counts[1])

'''

In [ ]:
'''
# Create an array with colors
colors = ["#6495ed", "#ff6347"]
# Set custom color palette
sns.set_palette(sns.color_palette(colors))

# create a column for more informative leend plotting
df.loc[df['output'] == 1, 'Rehospitalization'] = 'Yes'
df.loc[df['output'] == 0, 'Rehospitalization'] = 'No'

'''

In [ ]:


#df2.reset_index(inplace=True)
#df2 = df2.rename(columns = {'index':'id'})
#df2['id'] =df2['id'] + 1

In [ ]:
'''
# Recode sexual orientation
df2['sexorient'] = df2['sexorient'].replace(['Bisexual','Gay Male','Transgender','Lesbian', 'Other','Questioning', 'Transgender'],'LGBTQ+')
df2['sexorient'] = df2['sexorient'].replace(['Decline to State','Deferred','Missing'],'Missing/Not Reported')
print(df2[['client_id','sexorient']].groupby('client_id').max().sexorient.value_counts())

'''

In [ ]:
'''
#df2 = original.copy()
df2 = original.where(~mask, Xtrain)
'''